# Algoritmo de Seleção de Endereços - Projeto Uber


## Objetivo
O objetivo deste algoritmo é gerar um conjunto de pares de endereços na cidade de São Paulo para o treinamento de um modelo de precificação de corridas de Uber.

## Metodologia
O algoritmo terá natureza estocástica, dando preferência para endereços de chegada em distritos com maior densidade populacional e renda média por habitante (fatores relevantes para o uso do aplicativo Uber).

Para os endereços de saída, o algoritmo usará uma função de decaimento exponencial $P(d)$ (descrita abaixo) para gerar uma distância máxima para o endereço do destino, onde escolherá o endereço de destino uniformemente dentro desse intervalo.


Usaremos o conjunto de dados geográficos (disponível como um arquivo `.shape` com um grafo representando as ruas de São Paulo) distribuído pela plataforma [GeoSampa](https://geosampa.prefeitura.sp.gov.br) da Prefeitura de São Paulo para acessar os endereços disponíveis dentro da distância $d$ da origem.

## WIP

TODOs:
- Unificar .shape de Logradouros e Distritos
- 

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

### Implementação em Código Python

A implementação segue as etapas principais descritas abaixo:

1. __Carregamento e Preparação dos Dados:__

   - Utilizaremos a biblioteca `geopandas` para manipular arquivos `.shapefile`.
   - 
2. __Cálculo das Probabilidades de Escolha:__

   - Importe os dados do IBGE utilizando `pandas`.
   - Calcularemos a probabilidade de um distrito $\bar{d}$ com base na população e renda dos distritos.
   $$
   \text{Probabilidade}(\bar{d}) = \frac{\text{População}(\bar{d}) \cdot \text{RendaMédia}(\bar{d})}{\sum^{distritos}_{d} (\text{População}(d) \cdot \text{RendaMédia}(d))}
   $$
3. __Seleção do Endereço de Origem:__

   - Escolha aleatória de uma rua com `numpy.random.choice`.
   - Geração de um número dentro do intervalo válido.
4. __Seleção do Endereço de Destino:__

   - Use $P(r)$ para calcular o raio máximo:

     $$
     P(r) = \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{r^2}{2\sigma^2}}
     $$
   - Escolha uniformemente dentro do raio usando coordenadas do grafo.
5. **Visualização dos Resultados:**

   - Use `matplotlib` ou `folium` para criar mapas interativos.





### Visualização e Validação

Após a implementação, as corridas simuladas podem ser visualizadas em mapas interativos:

- **Mapas com `folium`:** Representam pares de origem e destino.
- **Gráficos:** Analisam estatisticamente a distribuição das distâncias e destinos.

Além disso, as corridas geradas podem ser exportadas para arquivos `.csv` para uso em modelos de treinamento.

# Setup

## Importar bibliotecas
Importamos todas as bibliotecas Python necessárias

In [21]:
# Manipulação de dados
import pandas as pd
import numpy as np
import geopandas as gpd

# Opções para exibição de dados
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualização de dados
import matplotlib as plt
    


# Importação de Dados
Importaremos todos os dados necessários para a análise.

In [66]:
# Carregar os dados do IBGE
df_pop = pd.read_csv('data/population_parameters_rates_by_district.csv')
df_renda = pd.read_csv('data/renda_media_distrito.csv')

# Carregar os dados do geo-dataframe da cidade de São Paulo
gdf = gpd.read_file('/home/sasinhe/uber/data/mapas/SaoPaulo_merged/sao_paulo.shp')



# Processamento de Dados

In [42]:
# TRATAMENTO DOS ARQUIVOS DO IBGE
# O resultado final deve ser um dataframe *df_prob* com os nomes dos distritos e suas respectivas probabilidades
#############################################################


# Remover espaços em branco e converter para minúsculas
df_pop['Nome_distrito'] = df_pop['Nome_distrito'].str.strip().str.lower()
df_renda['Nome_distrito'] = df_renda['Nome_distrito'].str.strip().str.lower()

# Definir intervalos de renda e seus pontos médios
income_intervals = {
    'Menos de 2 SM': 1,
    'De 2 a Menos de 5 SM': 3.5,
    'De 5 a Menos de 10 SM': 7.5,
    'De 10 a Menos de 15 SM': 12.5,
    'De 15 a Menos de 25 SM': 20,
    'De 25 SM e Mais ': 30
}

# Calcular a renda média para cada distrito
for col, midpoint in income_intervals.items():
    df_renda[col] = df_renda[col]/100 * midpoint


df_renda['average_income'] = df_renda[list(income_intervals.keys())].sum(axis=1)

# Normalizar a renda média
min_income = df_renda['average_income'].min()
max_income = df_renda['average_income'].max()
df_renda['normalized_income'] = (df_renda['average_income'] - min_income) / (max_income - min_income)

# Normalizar a população
min_pop = df_pop['Pop_2020'].min()
max_pop = df_pop['Pop_2020'].max()
df_pop['normalized_pop'] = (df_pop['Pop_2020'] - min_pop) / (max_pop - min_pop)

# Mesclar os dois dataframes na coluna 'Nome_distrito'
df_prob = pd.merge(df_pop, df_renda, on='Nome_distrito')

# Calcular a pontuação composta (média simples das normalizações)
df_prob['composite_score'] = (df_prob['normalized_income'] + df_prob['normalized_pop']) / 2

# Normalizar a pontuação composta para obter probabilidades
total_score = df_prob['composite_score'].sum()
df_prob['Probabilidade'] = df_prob['composite_score'] / total_score

# Criar um novo dataframe com os resultados para checagem
# result = df.copy()

# Remover colunas desnecessárias
df_prob = df_prob[["Nome_distrito", "Probabilidade"]]

# Exibir o resultado final com as probabilidades
df_prob

,Nome_distrito,Probabilidade
0,agua rasa,0.008669
1,alto de pinheiros,0.017343
2,anhanguera,0.005322
3,aricanduva,0.005759
4,artur alvim,0.005539
...,...,...
90,vila medeiros,0.007000
91,vila prudente,0.008727
92,vila sonia,0.014399
93,sao domingos,0.007517


In [70]:
# TRATAMENTO DOS ARQUIVOS SHAPE
# O resultado final deve ser um geo-dataframe *df_mapa* com as informações de cada logradouro
#############################################################

gdf.rename(columns={'ds_nomeds_': 'nome_distrito', 'lg_nome': 'nome_logradouro'}, inplace=True) # Renomear colunas
gdf = gdf.loc[gdf['lg_tipo'].isin(['R', 'AV'])] # Filtrar apenas ruas e avenidas
gdf['lg_tipo'] = gdf['lg_tipo'].map({'R' : 'Rua', 'AV': 'Avenida'})




,lg_ordem,lg_id,lg_seg_id,lg_or_geom,lg_codlog,lg_tipo,lg_titulo,lg_prep,nome_logradouro,lg_ini_par,lg_fim_par,lg_ini_imp,lg_fim_imp,nome_distrito,geometry
0,2.0,39752.0,122473,None,100293,R,ENG,None,JEAN BUFF,186.0,294.0,187.0,295.0,MANDAQUI,"LINESTRING (333470.822 7402866.037, 333369.621..."
1,1.0,35609.0,121965,None,059609,R,PROF,None,DJALMA BITAR,0.0,54.0,1.0,55.0,MANDAQUI,"LINESTRING (332339.366 7403364.492, 332332.73 ..."
2,140.0,33006.0,119557,None,038130,AV,ENG,None,CAETANO ALVARES,4516.0,4732.0,NaN,NaN,MANDAQUI,"LINESTRING (332246.383 7401626.527, 332118.298..."
3,2.0,35730.0,121049,None,060810,R,None,None,DORA,0.0,56.0,1.0,57.0,MANDAQUI,"LINESTRING (332513.907 7402140.596, 332485.705..."
4,1.0,57762.0,122081,None,279854,R,PDE,None,JOSE ALLAMANO,0.0,122.0,1.0,123.0,MANDAQUI,"LINESTRING (332465.693 7403799.133, 332380.156..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222525,24.0,65483.0,162737,None,358290,AV,None,None,MOREIRA GUIMARAES,NaN,NaN,361.0,481.0,None,"LINESTRING (331095.852 7387843.986, 331141.145..."
222526,17.0,44811.0,273405,None,150479,R,None,None,ORFANATO,1100.0,1146.0,1101.0,1147.0,None,"LINESTRING (339184.338 7391652.41, 339195.525 ..."
222528,23.0,51364.0,47027,None,215147,AV,None,None,ROQUE PETRONI JUNIOR,1410.0,1430.0,1411.0,1431.0,None,"LINESTRING (326576.267 7386790.661, 326586.24 ..."
222529,NaN,86960.0,308006,None,679631,R,None,None,CAQUEMON,NaN,NaN,NaN,NaN,None,"LINESTRING (319890.902 7386504.209, 319903.27 ..."


In [56]:
# Seleção do distrito de origem a partir do dataframe de probabilidades 'df_prob'

def selecionar_distrito_de_origem(prob_distrito_de_origem=df_prob, geodataframe=gdf):
    distritos = prob_distrito_de_origem['Nome_distrito']
    probabilidades = prob_distrito_de_origem['Probabilidade']
    distrito_amostrado = np.random.choice(distritos, p=probabilidades)
    print(f"Distrito amostrado: {distrito_amostrado}")
    df_ruas = geodataframe[geodataframe['distrito'] == distrito_amostrado]
    return df_ruas

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2

In [57]:
df_ruas = selecionar_distrito_de_origem()

Distrito amostrado: alto de pinheiros


KeyError: 'distrito'